In [1]:
import pandas
import numpy

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap as Basemap
from matplotlib.colors import rgb2hex, Normalize
from matplotlib.patches import Polygon
from matplotlib.colorbar import ColorbarBase

from bokeh.io import curdoc, push_notebook
from bokeh.layouts import layout, widgetbox
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.palettes import Magma256 as palette
from bokeh.models.widgets import Slider, Select
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper,
    ColorMapper
)

df = pandas.read_csv('openpv_all.csv',engine='python')


In [2]:
output_notebook()
us_states = numpy.load('us_states.npy').item()
df['year'] = df['date_installed'].str.slice(-4)
df['count'] = 1
df['rebates']=df['rebate'].str.strip('$').str.replace(',','').astype(float)

del us_states["HI"]
del us_states["AK"]
state_xs = [us_states[code]["lons"] for code in us_states]
state_ys = [us_states[code]["lats"] for code in us_states]
mod_df = df.copy()
source = ColumnDataSource(data=dict(
    x=state_xs,
    y=state_ys,
    name=us_states.keys(),
    data=[mod_df[
        (mod_df['year'] >= str(1999)) &
        (mod_df['year'] <= str(2015)) 
        ].groupby('state').mean()['annual_insolation'].replace(numpy.nan,0.)[key] for key in us_states.keys()],
    ))

def modify_plot(doc):

    TOOLS = "pan,wheel_zoom,reset,hover,save"
    #TOOLTIPS=[
    #    ("Title", "@title"),
    #    ("Year", "@year"),
    #    ("$", "@revenue")
    #]

    p = figure(title='', tools=TOOLS, plot_width=770, plot_height=490)
    color_mapper = LogColorMapper(palette=palette)
    
    p.patches('x', 'y', source=source,
        fill_color={'field': 'data', 'transform': color_mapper},
        fill_alpha=1.0, line_color="#884444", line_width=1.0)

    axis_map = {
        "Insolation": "annual_insolation",
        "Photovoltaic Installations": "count",
        "Photovoltaic Investment": "cost",
        "Photovoltaic Production": "annual_PV_prod",
        "Rebate Investment": "rebates",
        "Reported Production": "reported_annual_energy_prod",
        }

    slide_year = Slider(title="Year", start=2000, end=2015, value=2015, step=1)
    range_year = Slider(title="(+/-) Years", start=0, end=20, value=0, step=1)
    selector = Select(title="Data", options=sorted(axis_map.keys()), value='Insolation')
    
    def selections():
        select_val = axis_map[selector.value]
        hover_title = selector.value

        #print select_val
        if select_val == 'annual_insolation':
            selected = mod_df[
                (mod_df['year'] >= str(slide_year.value-range_year.value)) &
                (mod_df['year'] <= str(slide_year.value+range_year.value)) 
                ].groupby('state').mean().fillna(0)[select_val]
        else:
            selected = mod_df[
                (mod_df['year'] >= str(slide_year.value-range_year.value)) &
                (mod_df['year'] <= str(slide_year.value-range_year.value)) 
                ].groupby('state').sum().fillna(0)[select_val]
            
        return selected
    
    def gen_data(s_df):
        thisdata = []
        for key in us_states.keys():
            try:
                thisdata.append(s_df[key])
            except:
                thisdata.append(0)
        return thisdata
    
    def update():
        state_df = selections()
        
#        p.title.text = "%s from %s to %s" % (selector.value,min_year.value,max_year.value)
        source.data = dict(
            x=state_xs,
            y=state_ys,
            name=us_states.keys(),
#            data=[state_df[key] if state_df[key] else 0 for key in us_states.keys() if state_df[key] is not None],
            data = gen_data(state_df)
        )
#        print (source.data['name'],source.data['data'])

    controls = [selector, slide_year, range_year]
    for control in controls:
        control.on_change('value', lambda attr, old, new: update())

    sizing_mode = 'fixed'  # 'scale_width' also looks nice with this example
    hover_title = selector.value

    inputs = widgetbox(*controls, sizing_mode=sizing_mode)
    l = layout([
        [inputs,p],
    ], sizing_mode=sizing_mode)
    hover = p.select_one(HoverTool)
    hover.point_policy = "follow_mouse"
    hover.tooltips = [
            ("Name", "@name"),
            (hover_title, "@data"),
            ("(Long, Lat)", "($x, $y)"),
            ]
    
    
    doc.add_root(l)
#    slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
#    slider.on_change('value', callback)


#show(modify_plot)

Loading BokehJS ...

In [3]:
show(modify_plot)

ERROR:tornado.application:Uncaught exception GET /autoload.js?bokeh-autoload-element=2b3bb804-b420-462d-a8e0-13c4e7cdded4&bokeh-absolute-url=http://localhost:51839&resources=none (127.0.0.1)
HTTPServerRequest(protocol='http', host='localhost:51839', method='GET', uri='/autoload.js?bokeh-autoload-element=2b3bb804-b420-462d-a8e0-13c4e7cdded4&bokeh-absolute-url=http://localhost:51839&resources=none', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda2\lib\site-packages\tornado\web.py", line 1543, in _execute
    result = yield result
  File "C:\ProgramData\Anaconda2\lib\site-packages\tornado\gen.py", line 1099, in run
    value = future.result()
  File "C:\ProgramData\Anaconda2\lib\site-packages\tornado\concurrent.py", line 260, in result
    raise_exc_info(self._exc_info)
  File "C:\ProgramData\Anaconda2\lib\site-packages\tornado\gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "C:\ProgramData\Anaconda2\l